## Window Functions

Window functions allow you to maintain the values of your original table while displaying grouped or summative information alongside in another column. This is why many Data Scientists and Data Engineers love to use window functions for complex data analysis.

Example:
```sql
SELECT 
   month,
   change_in_followers,
   SUM(change_in_followers) OVER (
      ORDER BY month
   ) AS 'running_total'
FROM
   social_media
WHERE
   username = 'instagram';
```
- `SELECT month, change_in_followers`: Same as usual, selecting the columns.
- `SUM(change_in_followers)`: Here is our aggregate function to find the SUM of our chosen column.
- `OVER`: This is the clause that designates SUM as a window function.
- `ORDER BY month`: Here we declare what we would like our window function to do.
    - This window function is taking the sum of followers for each month.
    - So for each month, the window function adds the current month’s change_in_followers to our running total.
- Then name the running total column `'running_total'`.
- And lastly, this is all coming from table `social_media` where the username is instagram.

`PARTITION BY` is a subclause of the `OVER` clause and divides a query’s result set into parts. It’s very similar to `GROUP BY` except it does not reduce the number of rows returned.

While using `GROUP BY` only allows one row to be returned for each group, `PARTITION BY` allows you to see all of the resultant rows.

Example:
```sql
SELECT
   username,
   posts,
   FIRST_VALUE (posts) OVER (
      PARTITION BY username 
      ORDER BY posts
   ) fewest_posts
FROM
   social_media;
```
- `FIRST_VALUE()` returns the first value in an ordered set of values.
- `LAST_VALUE()` returns the last value in an ordered set of values.
- This query should look familiar overall as it follows the standard window function format, however, we are using `FIRST_VALUE` now for `posts`. This means our window function will pull the first value from the `posts` column.
- `OVER (PARTITION BY username ORDER BY posts) fewest_posts`: here we can see that posts is going to be pulled based on `username` due to the `PARTITION BY`. We are naming this column `fewest_posts` because of the `ORDER BY` which defaults to ascending order.
- And all of this is coming from our `social_media` table.


Example:
```sql
SELECT
   username,
   posts,
   LAST_VALUE (posts) OVER (
      PARTITION BY username 
      ORDER BY posts
      RANGE BETWEEN UNBOUNDED PRECEDING AND 
      UNBOUNDED FOLLOWING
    ) most_posts
FROM
    social_media;
```
`RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING` specifies the frame for our window function as the current partition and thus returns the highest number of posts in one month for each user.

Window functions can use `LAG` or `LEAD` in order to access information from a row at a specified offset which comes before (`LAG`) or after (`LEAD`) the current row.

`LAG` takes up to three arguments:
- column (required)
- offset (optional, default 1 row offset)
- default (optional, what to replace default null values with)

Example:
```sql
SELECT
   artist,
   week,
   streams_millions,
   LAG(streams_millions, 1, 0) OVER (
      ORDER BY week 
   ) previous_week_streams 
FROM
   streams 
WHERE
   artist = 'Lady Gaga';
```

Example: `ROW_NUMBER()`
```sql
SELECT 
   ROW_NUMBER() OVER (
      ORDER BY streams_millions
   ) AS 'row_num', 
   artist, 
   week,
   streams_millions
FROM
   streams;
```

Example: `RANK()`
```sql
SELECT 
   RANK() OVER (
      ORDER BY streams_millions
   ) AS 'rank', 
   artist, 
   week,
   streams_millions
FROM
   streams;
```

Example: `NTILE()`
```sql
SELECT 
   NTILE(5) OVER (
      ORDER BY streams_millions DESC
   ) AS 'weekly_streams_group', 
   artist, 
   week,
   streams_millions
FROM
   streams;
```